In [1]:
import pandas as pd
import numpy as np
import os
path = os.getcwd();
filename = os.listdir(f"{path}/textfile/");
ymlfile = os.path.join(path, "DOTAv2.ymal")
df_meta = pd.read_csv(f"{path}/textfile/{filename[0]}", encoding='utf_8_sig')
df_meta, ymlfile

(          m_map  side                                           filepath  \
 0    56845-1B_K    A面             .\Image\56845-1B_K\NG-A面\A-唇尖缺料-01.jpg   
 1    56845-1B_K    A面             .\Image\56845-1B_K\NG-A面\A-外徑死料-01.jpg   
 2    56845-1B_K    A面             .\Image\56845-1B_K\NG-A面\A-外徑毛邊-01.jpg   
 3    56845-1B_K    A面             .\Image\56845-1B_K\NG-A面\A-外徑毛邊-02.jpg   
 4    56845-1B_K    A面             .\Image\56845-1B_K\NG-A面\A-外徑毛邊-03.jpg   
 ..          ...   ...                                                ...   
 304  68620-1E_K    B面  .\Image\68620-1E_K\OK-B面\Src-16-Time-13-59-52-...   
 305  68620-1E_K    B面  .\Image\68620-1E_K\OK-B面\Src-17-Time-14-00-00-...   
 306  68620-1E_K    B面  .\Image\68620-1E_K\OK-B面\Src-18-Time-14-00-10-...   
 307  68620-1E_K    B面  .\Image\68620-1E_K\OK-B面\Src-19-Time-14-00-25-...   
 308  68620-1E_K    B面  .\Image\68620-1E_K\OK-B面\Src-20-Time-14-00-36-...   
 
                                  rename  
 0     .\dotadataset\56845-1B_K

In [3]:
filelist = os.listdir(os.path.join(path,"dotadataset"))
# csvlist = [file for file in filelist if (file.find(".csv") != -1 and file.find("B面") != -1)]
csvlist = [file for file in filelist if (file.find(".csv") != -1)]
csvlist_df = pd.DataFrame(csvlist)
# imglist = [file for file in filelist if (file.find(".jpg") != -1 and file.find("B面") != -1)]
imglist = [file for file in filelist if (file.find(".jpg") != -1)]


filelist_df = pd.DataFrame([imglist, csvlist]).T
filelist_df.columns = ["jpg", "csv"]



train = filelist_df.sample(frac=0.8)
test_and_val = filelist_df.drop(train.index)
test = test_and_val.sample(frac=0.5)
val = test_and_val.drop(test.index)
filelist_df

,jpg,csv
0,56845-1B_K_0.jpg,56845-1B_K_0.csv
1,56845-1B_K_1.jpg,56845-1B_K_1.csv
2,56845-1B_K_10.jpg,56845-1B_K_10.csv
3,56845-1B_K_11.jpg,56845-1B_K_11.csv
4,56845-1B_K_12.jpg,56845-1B_K_12.csv
...,...,...
655,68620-1E_K_FlipY_5.jpg,68620-1E_K_FlipY_5.csv
656,68620-1E_K_FlipY_6.jpg,68620-1E_K_FlipY_6.csv
657,68620-1E_K_FlipY_7.jpg,68620-1E_K_FlipY_7.csv
658,68620-1E_K_FlipY_8.jpg,68620-1E_K_FlipY_8.csv


In [9]:
from PIL import Image
import cv2
import sys
import shutil
abspath = os.path.join(path,"dotadataset")
list_data = {"train":train,
             "test":test,
             "val":val}

def rotate_rect_to_yolov8obb(rotate_rect):
    x1, y1, x2, y2, x3, y3, x4, y4, category, difficult = rotate_rect

    # 計算中心點座標
    cx = (x1 + x2 + x3 + x4) / 4
    cy = (y1 + y2 + y3 + y4) / 4

    # 計算座標點
    x1_rel = x1 - cx
    y1_rel = y1 - cy
    x2_rel = x2 - cx
    y2_rel = y2 - cy
    x3_rel = x3 - cx
    y3_rel = y3 - cy
    x4_rel = x4 - cx
    y4_rel = y4 - cy

    # YOLOv8 OBB 格式: [category, x1, y1, x2, y2, x3, y3, x4, y4, difficult]
    yolov8obb_label = [category, x1_rel, y1_rel, x2_rel, y2_rel, x3_rel, y3_rel, x4_rel, y4_rel, difficult]

    return yolov8obb_label

try:
    os.mkdir(os.path.join(os.path.join(path, "images"), "train"))
    os.mkdir(os.path.join(os.path.join(path, "images"), "test"))
    os.mkdir(os.path.join(os.path.join(path, "images"), "val"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "train"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "test"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "val"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "train_original"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "test_original"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "val_original"))
except:
    pass


path = os.getcwd();
for rmfolder in list_data.keys():
    print(rmfolder)
    shutil.rmtree(os.path.join(os.path.join(path, "images"), rmfolder))
for rmfolder in list_data.keys():
    print(rmfolder)
    if (rmfolder.find("origin")):
        shutil.rmtree(os.path.join(os.path.join(path, "labels"), rmfolder))
try:
    os.mkdir(os.path.join(os.path.join(path, "images"), "train"))
    os.mkdir(os.path.join(os.path.join(path, "images"), "test"))
    os.mkdir(os.path.join(os.path.join(path, "images"), "val"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "train"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "test"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "val"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "train_original"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "test_original"))
    os.mkdir(os.path.join(os.path.join(path, "labels"), "val_original"))
except:
    pass

    
# cate = {
#     "DC4S":0,
#     "NAK":1,
#     "ItemSize":2,
#     "Others":3,
# }
cate = {
    "ItemContent":0,
    "Others":1
}
for dir in list_data.keys():
    dirpath = os.path.join(os.path.join(path, "images"), dir)
    for filename in list_data[dir]["jpg"]:
        print(abspath)
        print(filename)
        
        img = Image.open(os.path.join(abspath, filename))
        img.save(os.path.join(dirpath, filename))
    dirpath = os.path.join(os.path.join(path, "labels"), f"{dir}_original")
    for pdfile in list_data[dir]["csv"]:
        txtfilename = pdfile.replace("csv","txt")
        df = pd.read_csv(os.path.join(abspath, pdfile))
        df.iloc[:, :8] = df.iloc[:, :8].astype(float)
        df.iloc[:, -1] = df.iloc[:, -1].astype(int)
        # print(df)
        # sys.exit()
        df.to_csv(os.path.join(dirpath, txtfilename), sep=' ', index=False, header=None)
    # dirpath = os.path.join(os.path.join(path, "labels"), f"{dir}_original")
    # for pdfile in list_data[dir]["csv"]:
    #     txtfilename = pdfile.replace("csv","txt")
    #     df = pd.read_csv(os.path.join(abspath, pdfile))
    #     df.iloc[:, :8] = df.iloc[:, :8].astype(float)
    #     df.iloc[:, -1] = df.iloc[:, -1].astype(int)

    #     df.replace({"category":cate}, inplace=True)
    #     cate_df = df.pop("category")
        

    #     new_df = cate_df.to_frame()
        
    #     df.insert(0, "category", cate_df)
        

    #     df.to_csv(os.path.join(dirpath, txtfilename), sep=' ', index=False, header=None)
    print(dir+" complete...")


train
test
val
train
test
val
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
68620-1E_K_FlipX_34.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
68620-1E_K_Bright_5.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
57634-1C_K_Bright_10.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
56845-1B_K_FlipY_12.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
57634-1C_K_Bright_26.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
57634-1C_K_20.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
56845-1B_K_FlipX_3.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
67400-1A_K_FlipX_31.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
57634-1C_K_FlipY_25.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
57634-1C_K_FlipX_3.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
56845-1B_K_FlipY_2.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset
56845-1B_K_FlipX_40.jpg
c:\Users\User\Desktop\C_VSCode\Yolov8OBB\dotadataset


In [15]:
import os
from pathlib import Path
from tqdm import tqdm 
import cv2
from PIL import Image

def convert_dota_to_yolo_obb(dota_root_path: str):
    """
    Converts DOTA dataset annotations to YOLO OBB (Oriented Bounding Box) format.

    The function processes images in the 'train' and 'val' folders of the DOTA dataset. For each image, it reads the
    associated label from the original labels directory and writes new labels in YOLO OBB format to a new directory.

    Args:
        dota_root_path (str): The root directory path of the DOTA dataset.

    Example:
        ```python
        from ultralytics.data.converter import convert_dota_to_yolo_obb

        convert_dota_to_yolo_obb('path/to/DOTA')
        ```

    Notes:
        The directory structure assumed for the DOTA dataset:
            - DOTA
                ├─ images
                │   ├─ train
                │   └─ val
                └─ labels
                    ├─ train_original
                    └─ val_original

        After execution, the function will organize the labels into:
            - DOTA
                └─ labels
                    ├─ train
                    └─ val
    """
    dota_root_path = Path(dota_root_path)

    # Class names to indices mapping
    class_mapping = {
        "ItemContent":0,
        "Others":1
    }

    def convert_label(image_name, image_width, image_height, orig_label_dir, save_dir):
        """Converts a single image's DOTA annotation to YOLO OBB format and saves it to a specified directory."""
        orig_label_path = orig_label_dir / f'{image_name}.txt'
        save_path = save_dir / f'{image_name}.txt'

        with orig_label_path.open('r') as f, save_path.open('w') as g:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) < 9:
                    continue
                class_name = parts[8]
                class_idx = class_mapping[class_name]
                coords = [float(p) for p in parts[:8]]
                normalized_coords = [
                    coords[i] / image_width if i % 2 == 0 else coords[i] / image_height for i in range(8)]
                formatted_coords = ['{:.6g}'.format(coord) for coord in normalized_coords]
                g.write(f"{class_idx} {' '.join(formatted_coords)}\n")

    for phase in ['train', 'val']:
        image_dir = dota_root_path / 'images' / phase
        orig_label_dir = dota_root_path / 'labels' / f'{phase}_original'
        save_dir = dota_root_path / 'labels' / phase

        save_dir.mkdir(parents=True, exist_ok=True)

        image_paths = list(image_dir.iterdir())
        for image_path in tqdm(image_paths, desc=f'Processing {phase} images'):
            if image_path.suffix != '.jpg':
                continue
            image_name_without_ext = image_path.stem
            img = cv2.imread(str(image_path))
            if (img is None):
                img = Image.open(str(image_path))
                h = img.width
                w = img.height
            else:
                h, w = img.shape[:2]
            convert_label(image_name_without_ext, w, h, orig_label_dir, save_dir)

In [16]:


root_path = os.getcwd()
root_path = os.path.join(root_path, "NAK")
root_path = os.path.join(root_path, "datasets")
os.listdir(root_path)
convert_dota_to_yolo_obb(root_path)

Processing val images: 100%|██████████| 66/66 [00:01<00:00, 39.60it/s]
